In [69]:
import pandas as pd
import ast

***Read bangalore excel file***

In [70]:
bangalore = pd.read_excel("../raw_data/bangalore_cars.xlsx")

In [71]:
bangalore.head()

,new_car_detail,new_car_overview,new_car_feature,new_car_specs,car_links
0,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
1,"{'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
2,"{'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...
3,"{'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/buy-used-car-details/...
4,"{'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...","{'heading': 'Car overview', 'top': [{'key': 'R...","{'heading': 'Features', 'top': [{'value': 'Pow...","{'heading': 'Specifications', 'top': [{'key': ...",https://www.cardekho.com/used-car-details/used...


***Cleaned new car overview***

In [72]:

bangalore['new_car_overview'] = bangalore['new_car_overview'].apply(ast.literal_eval)

df_expanded = pd.json_normalize(bangalore['new_car_overview'])


In [73]:
rows = []
for index, row in df_expanded.iterrows():
    # Extract the 'top' list from the row
    top_list = row['top']
    
    # Create a dictionary for the current row
    columns_dict = {item['key']: item['value'] for item in top_list}
    
    # Append the dictionary to the list of rows
    rows.append(columns_dict)

car_overview = pd.DataFrame(rows)


In [74]:
car_overview.head()

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2015,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",KA51,Third Owner,998 cc,Manual,2015.0
1,Feb 2018,Comprehensive,Petrol,5 Seats,"32,706 Kms",KA05,Second Owner,1497 cc,Manual,2018.0
2,Sept 2018,Comprehensive,Petrol,5 Seats,"11,949 Kms",KA03,First Owner,1199 cc,Manual,2018.0
3,Dec 2014,Comprehensive,Petrol,5 Seats,"17,794 Kms",KA53,First Owner,1197 cc,Manual,2014.0
4,2015,Third Party insurance,Diesel,5 Seats,"60,000 Kms",KA04,First Owner,1248 cc,Manual,2015.0


***Cleaned new car features***

In [75]:
bangalore['new_car_feature'] = bangalore['new_car_feature'].apply(ast.literal_eval)

features_expanded = pd.json_normalize(bangalore['new_car_feature'])

In [76]:
# Expand 'top' field
def expand_top(top_list):
    return [item['value'] for item in top_list]

# Extract 'top' features
bangalore['top_features'] = bangalore['new_car_feature'].apply(lambda x: expand_top(x['top']))

# Expand 'data' field
def expand_data(data_list):
    expanded_data = {}
    for item in data_list:
        heading = item['heading']
        expanded_data[heading] = [feature['value'] for feature in item['list']]
    return expanded_data

# Extract 'data' features
data_features = bangalore['new_car_feature'].apply(lambda x: expand_data(x['data']))

# Normalize 'data_features' and concatenate with 'top_features'
data_features_df = pd.json_normalize(data_features)

# Concatenate 'top_features' and 'data_features_df'
top_feature_df = pd.concat([bangalore[['top_features']], data_features_df], axis=1)

In [77]:
top_feature_df.head()

,top_features,Comfort & Convenience,Interior,Exterior,Safety,Entertainment & Communication
0,"[Power Steering, Power Windows Front, Air Cond...","[Power Steering, Power Windows Front, Power Wi...","[Air Conditioner, Heater, Digital Odometer, El...","[Adjustable Head Lights, Manually Adjustable E...","[Centeral Locking, Child Safety Locks, Day Nig...",NaN
1,"[Power Steering, Power Windows Front, Air Cond...","[Power Steering, Power Windows Front, Power Wi...","[Air Conditioner, Heater, Adjustable Steering,...","[Adjustable Head Lights, Fog Lights Front, Pow...","[Anti Lock Braking System, Centeral Locking, P...","[Cd Player, Radio, Speakers Front, Speakers Re..."
2,"[Power Steering, Power Windows Front, Air Cond...","[Power Steering, Power Windows Front, Power Wi...","[Air Conditioner, Heater, Adjustable Steering,...","[Adjustable Head Lights, Fog Lights Front, Pow...","[Anti Lock Braking System, Centeral Locking, P...","[Cd Player, Radio, Speakers Front, Speakers Re..."
3,"[Power Steering, Power Windows Front, Air Cond...","[Power Steering, Power Windows Front, Power Wi...","[Air Conditioner, Heater, Adjustable Steering,...","[Adjustable Head Lights, Fog Lights Front, Pow...","[Anti Lock Braking System, Centeral Locking, P...","[Cd Player, Radio, Speakers Front, Speakers Re..."
4,"[Power Steering, Power Windows Front, Air Cond...","[Power Steering, Power Windows Front, Power Wi...","[Air Conditioner, Heater, Adjustable Steering,...","[Adjustable Head Lights, Fog Lights Front, Pow...","[Anti Lock Braking System, Centeral Locking, P...","[Radio, Audio System Remote Control, Speakers ..."


***Cleaned new car specs***

In [78]:
bangalore['new_car_specs'] = bangalore['new_car_specs'].apply(ast.literal_eval)

specs_expanded = pd.json_normalize(bangalore['new_car_specs'])

In [79]:
rows = []
for index, row in specs_expanded.iterrows():
    # Extract the 'top' list from the row
    top_list = row['top']
    
    # Create a dictionary for the current row
    columns_dict = {item['key']: item['value'] for item in top_list}
    
    # Append the dictionary to the list of rows
    rows.append(columns_dict)

car_specs = pd.DataFrame(rows)

In [80]:
car_specs.head()

,Mileage,Engine,Max Power,Torque,Seats,Wheel Size
0,23.1 kmpl,998 CC,67.04bhp,90Nm,5,NaN
1,17 kmpl,1497 CC,121.31bhp,150Nm,5,16
2,23.84 kmpl,1199 CC,84bhp,114Nm,5,14
3,19.1 kmpl,1197 CC,81.86bhp,113.75Nm,5,14
4,23.65 kmpl,1248 CC,88.5bhp,200Nm,5,16


***Cleaned new car details***

In [81]:
bangalore['new_car_detail'] = bangalore['new_car_detail'].apply(ast.literal_eval)

detail_expanded = pd.json_normalize(bangalore['new_car_detail'])

In [82]:
rows = []
for index, row in df_expanded.iterrows():
    # Extract the 'top' list from the row
    top_list = row['top']
    
    # Create a dictionary for the current row
    columns_dict = {item['key']: item['value'] for item in top_list}
    
    # Append the dictionary to the list of rows
    rows.append(columns_dict)

car_overview = pd.DataFrame(rows)

In [57]:
newcar_details = pd.DataFrame(detail_expanded)

In [58]:
newcar_details.head()

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Petrol,Hatchback,"1,20,000",Manual,3,3rd Owner,Maruti,Maruti Celerio,2015,3979,VXI,₹ 4 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,SUV,"32,706",Manual,2,2nd Owner,Ford,Ford Ecosport,2018,6087,1.5 Petrol Titanium BSIV,₹ 8.11 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,Hatchback,"11,949",Manual,1,1st Owner,Tata,Tata Tiago,2018,2983,1.2 Revotron XZ,₹ 5.85 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Sedan,"17,794",Manual,1,1st Owner,Hyundai,Hyundai Xcent,2014,1867,1.2 Kappa S Option,₹ 4.62 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
4,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Maruti,Maruti SX4 S Cross,2015,4277,DDiS 200 Zeta,₹ 7.90 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/ic...,Trending Car!,High chances of sale in next 6 days
